In [ ]:
# default_exp metrics

# Metrics

> Definition of the metrics that can be used to evaluate models

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
from fastai_object_detection.external.mean_average_precision_source import MetricBuilder
from fastai.metrics import Metric
from fastai.torch_basics import *
from fastai.torch_core import *
from functools import partial

In [ ]:
#export     
class mAP_Metric():
    "Metric to calculate mAP for different IoU thresholds"
    def __init__(self, iou_thresholds, recall_thresholds=None, mpolicy="greedy", name="mAP", remove_background_class=True):
        self.__name__ = name
        self.iou_thresholds = iou_thresholds
        self.recall_thresholds = recall_thresholds
        self.mpolicy = mpolicy
        self.remove_background_class = remove_background_class
        
    def __call__(self, preds, targs, num_classes):
        if self.remove_background_class:
            num_classes=num_classes-1
        #metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=True, num_classes=num_classes)
        metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=False, num_classes=num_classes)
        for sample_preds, sample_targs in self.create_metric_samples(preds, targs):
            metric_fn.add(sample_preds, sample_targs)
        metric_batch = metric_fn.value(iou_thresholds=self.iou_thresholds,
                                       recall_thresholds=self.recall_thresholds, 
                                       mpolicy=self.mpolicy)['mAP']
        return metric_batch
    
    def create_metric_samples(self, preds, targs):
        pred_samples = []
        for pred in preds:
            res = torch.cat([pred["boxes"], pred["labels"].unsqueeze(-1), pred["scores"].unsqueeze(-1)], dim=1) 
            pred_np = res.detach().cpu()#.numpy()
            if self.remove_background_class:
                # first idx is background
                try:
                    pred_np= pred_np-np.array([0,0,0,0,1,0])
                except: pass
            pred_samples.append(pred_np)

        targ_samples = []
        for targ in targs: # targs : yb[0]
            targ = torch.cat([targ["boxes"],targ["labels"].unsqueeze(-1)], dim=1)
            targ = torch.cat([targ, torch.zeros([targ.shape[0], 2], device=targ.device)], dim=1)
            targ_np = targ.detach().cpu()
            #targ_np = np.array(targ.detach().cpu())
            if self.remove_background_class:
                # first idx is background 
                try:
                    targ_np= targ_np-np.array([0,0,0,0,1,0,0])
                except: pass
            targ_samples.append(targ_np)

        return [s for s in zip(pred_samples, targ_samples)]
    

In [ ]:
#export 

class _AvgMetric_ObjectDetection(Metric):
    "Average the values of `func` taking into account potential different batch sizes"
    def __init__(self, func): self.func = func
    def reset(self): self.total,self.count = 0.,0
    def accumulate(self, learn):
        bs = len(learn.xb[0])
        self.total += learn.to_detach(self.func(learn.pred, *learn.yb, num_classes=len(learn.dls.vocab)))*bs
        self.count += bs
    @property
    def value(self): return self.total/self.count if self.count != 0 else None
    @property
    def name(self): return self.func.func.__name__ if hasattr(self.func, 'func') else  self.func.__name__
    
           


## Function to create mAP metrics

In [ ]:
#export 

def create_mAP_metric(iou_tresh, recall_thresholds, mpolicy, metric_name, remove_background_class=True):
    """ Creates a function to pass into learner for measuring mAP.
    iou_tresh: float or np.arange, f.e.: np.arange(0.5, 1.0, 0.05)
    recall_thresholds: None or np.arange, f.e.: np.arange(0., 1.01, 0.01)
    mpolicy: str, 'soft' or 'greedy'
    metric_name: str, name to display in fastai´s recorder
    remove_background_class: True or False, remove first index before evaluation, as it represents background class in our dataloader
    Metric Examples:
    COCO mAP: set recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy="soft"
    VOC PASCAL mAP: set recall_thresholds=np.arange(0., 1.1, 0.1), mpolicy="greedy"
    VOC PASCAL mAP in all points: set recall_thresholds=None, mpolicy="greedy"
    """
    return _AvgMetric_ObjectDetection(mAP_Metric(iou_tresh, recall_thresholds=recall_thresholds, mpolicy=mpolicy,
                                                    name=metric_name, remove_background_class=True)) 
   

In [ ]:
#export 
# coco mAP    
mAP_at_IoU40 = _AvgMetric_ObjectDetection(mAP_Metric(0.4, recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy="soft",
                                                    name="mAP@IoU>0.4", remove_background_class=True))
mAP_at_IoU50 = _AvgMetric_ObjectDetection(mAP_Metric(0.5, recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy="soft",
                                                    name="mAP@IoU>0.5", remove_background_class=True))
mAP_at_IoU60 = _AvgMetric_ObjectDetection(mAP_Metric(0.6, recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy="soft",
                                                    name="mAP@IoU>0.6", remove_background_class=True))
mAP_at_IoU70 = _AvgMetric_ObjectDetection(mAP_Metric(0.7, recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy="soft",
                                                    name="mAP@IoU>0.7", remove_background_class=True))
mAP_at_IoU80 = _AvgMetric_ObjectDetection(mAP_Metric(0.8, recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy="soft",
                                                    name="mAP@IoU>0.8", remove_background_class=True))
mAP_at_IoU90 = _AvgMetric_ObjectDetection(mAP_Metric(0.9, recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy="soft",
                                                    name="mAP@IoU>0.9", remove_background_class=True))
mAP_at_IoU50_95 = _AvgMetric_ObjectDetection(mAP_Metric(np.arange(0.5, 1.0, 0.05), recall_thresholds=np.arange(0., 1.01, 0.01), mpolicy="soft",
                                                    name="mAP@IoU 0.5:0.95", remove_background_class=True)) 

## Custom mAP metrics

First we create some predictions and targets. Note that our dataloader contains a background class with index 0 and all metrics remove by default the background class, so the first class has index 1 and the number of classes is 2.

In [ ]:
num_classes = 2

In [ ]:
boxes = torch.tensor([
    [439, 157, 556, 241],
    [437, 246, 518, 351],
    [515, 306, 595, 375],
    [407, 386, 531, 476],
    [544, 419, 621, 476],
    [609, 297, 636, 392]])
labels = torch.ones(6, dtype=torch.long)

targs = [dict({"boxes":boxes, "labels":labels})]
targs

[{'boxes': tensor([[439, 157, 556, 241],
          [437, 246, 518, 351],
          [515, 306, 595, 375],
          [407, 386, 531, 476],
          [544, 419, 621, 476],
          [609, 297, 636, 392]]),
  'labels': tensor([1, 1, 1, 1, 1, 1])}]

In [ ]:
boxes = torch.tensor([
    [429, 219, 528, 247],
    [433, 260, 506, 336],
    [518, 314, 603, 369],
    [592, 310, 634, 388],
    [403, 384, 517, 461],
    [405, 429, 519, 470],
    [433, 272, 499, 341],
    [413, 390, 515, 459]])
labels = torch.ones(8, dtype=torch.long)
scores = torch.tensor([0.460851, 0.269833, 0.462608, 0.298196, 0.382881, 0.369369, 0.272826, 0.619459])

preds = [dict({"boxes":boxes, "labels":labels, "scores":scores})]
preds

[{'boxes': tensor([[429, 219, 528, 247],
          [433, 260, 506, 336],
          [518, 314, 603, 369],
          [592, 310, 634, 388],
          [403, 384, 517, 461],
          [405, 429, 519, 470],
          [433, 272, 499, 341],
          [413, 390, 515, 459]]),
  'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1]),
  'scores': tensor([0.4609, 0.2698, 0.4626, 0.2982, 0.3829, 0.3694, 0.2728, 0.6195])}]

### VOC PASCAL

In [ ]:
voc_pascal = create_mAP_metric(0.5, np.arange(0., 1.1, 0.1), "greedy", "VOC PASCAL mAP", 
                               remove_background_class=True)
voc_pascal.func(preds, targs, num_classes=num_classes)

tensor(0.5000)

In [ ]:
voc_pascal_all_pnts = create_mAP_metric(0.5, None, "greedy", "VOC PASCAL mAP all points", 
                                        remove_background_class=True)
voc_pascal_all_pnts.func(preds, targs, num_classes=num_classes)

tensor(0.5000)

### COCO mAP

In [ ]:
coco_map_50 = create_mAP_metric(0.5, np.arange(0., 1.01, 0.01), "soft", "COCO mAP@0.5", 
                                remove_background_class=True)
coco_map_50.func(preds, targs, num_classes=num_classes)

tensor(0.5000)

In [ ]:
coco_map_50_95 = create_mAP_metric(np.arange(0.5, 1, .05), np.arange(0., 1.01, 0.01), "soft", "COCO mAP@[0.5:0.95]", 
                                remove_background_class=True)
coco_map_50_95.func(preds, targs, num_classes=num_classes)

tensor(0.1573)

In [ ]:
test_close(voc_pascal.func(preds, targs, num_classes=2), 0.5, eps=1e-03)
test_close(voc_pascal_all_pnts.func(preds, targs, num_classes=2), 0.5, eps=1e-03)
test_close(coco_map_50.func(preds, targs, num_classes=2), 0.5, eps=1e-03)
test_close(coco_map_50_95.func(preds, targs, num_classes=2), 0.157, eps=1e-03)

## Prebuilt metrics

There are some prebuilt metrics, which you can use instantly:

COCO mAP:
* `mAP_at_IoU40` 
* `mAP_at_IoU50` 
* `mAP_at_IoU60`
* `mAP_at_IoU70` 
* `mAP_at_IoU80`
* `mAP_at_IoU90` 
* `mAP_at_IoU50_95` (mAP@[0.50:0.95:0.05])